# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1]:
#Dependencies
import pandas as pd
import numpy as np
import os



In [2]:
# defining file path
schools_file = os.path.join('raw_data','schools_complete.csv')
students_file = os.path.join('raw_data', 'students_complete.csv')

# read school(sch) file
sch_df = pd.read_csv(schools_file)

#read student (stu) file
stu_df = pd.read_csv(students_file)

#renames for merge
sch_df.rename(columns = {'name': 'school'}, inplace = True)

merge_df = stu_df.merge(sch_df, how = 'left', on = 'school')

merge_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [3]:
#creating an array of unique school names
unique_school_names = sch_df['school'].unique()

#gives the length of unique school names to give us how many schools
unique_school_count = len(unique_school_names)

#district student count
dist_student_count = sch_df['size'].sum()

#student count from student file (to verify with district student count)
total_student_rec = stu_df['name'].count()

#total budget
total_budget = sch_df['budget'].sum()

#calculations for number and % passing reading
num_passing_reading = stu_df.loc[stu_df['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = num_passing_reading/total_student_rec

#calculations for number and % passing math
num_passing_math = stu_df.loc[stu_df['math_score'] >= 70]['math_score'].count()
perc_pass_math = num_passing_math/total_student_rec

#average math score calculation
avg_math_score = stu_df['math_score'].mean()

#average reading score calculation
avg_reading_score = stu_df['reading_score'].mean()

#Overall Passing Rate Calculations
overall_pass = stu_df[(stu_df['math_score'] >= 70) & (stu_df['reading_score'] >= 70)]['name'].count()/total_student_rec

# district dataframe from dictionary

district_summary = pd.DataFrame({
    
    "Total Schools": [unique_school_count],
    "Total Students": [dist_student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[perc_pass_reading],
    "% Passing Math": [perc_pass_math],
    "Overall Passing Rate": [overall_pass]

})

#store as different df to change order
dist_sum = district_summary[["Total Schools", 
                             "Total Students", 
                             "Total Budget", 
                             "Average Reading Score", 
                             "Average Math Score", 
                             '% Passing Reading', 
                             '% Passing Math', 
                             'Overall Passing Rate']]

#format cells
dist_sum.style.format({"Total Budget": "${:,.2f}", 
                       "Average Reading Score": "{:.1f}", 
                       "Average Math Score": "{:.1f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,"$24,649,428.00",81.9,79.0,85.8%,75.0%,65.2%


## School Summary

In [4]:
#groups by school
by_school = merge_df.set_index('school').groupby(['school'])

#school types
sch_types = sch_df.set_index('school')['type']

# total students by school
stu_per_sch = by_school['Student ID'].count()

# school budget
sch_budget = sch_df.set_index('school')['budget']

#per student budget
stu_budget = sch_df.set_index('school')['budget']/sch_df.set_index('school')['size']

#avg scores by school
average_math = by_school['math_score'].mean()
average_read = by_school['reading_score'].mean()

# % passing scores
pass_math = merge_df[merge_df['math_score'] >= 70].groupby('school')['Student ID'].count()/stu_per_sch 
pass_read = merge_df[merge_df['reading_score'] >= 70].groupby('school')['Student ID'].count()/stu_per_sch 
overall = merge_df[(merge_df['reading_score'] >= 70) & (merge_df['math_score'] >= 70)].groupby('school')['Student ID'].count()/stu_per_sch 

sch_summary = pd.DataFrame({
    "School Type": sch_types,
    "Total Students": stu_per_sch,
    "Per Student Budget": stu_budget,
    "Total School Budget": sch_budget,
    "Average Math Score": average_math,
    "Average Reading Score": average_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})




#munging
sch_summary = sch_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

#formatting
sch_summary.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Average Math Score': "{:.1f}", 
                          'Average Reading Score': "{:.1f}", 
                          "% Passing Math": "{:.1%}", 
                          "% Passing Reading": "{:.1%}", 
                          "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928",$628,77.0,81.0,66.7%,81.9%,54.6%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,91.3%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,53.2%
Ford High School,District,"2,739","$1,763,916",$644,77.1,80.7,68.3%,79.3%,54.3%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,90.6%
Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,53.5%
Holden High School,Charter,427,"$248,087",$581,83.8,83.8,92.5%,96.3%,89.2%
Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.1,81.0,66.1%,81.2%,53.5%
Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,90.5%


## Top Performing Schools (By Passing Rate)

In [5]:
top_5 = sch_summary.sort_values("Overall Passing Rate", ascending = False)
top_5.head().style.format({'Total Students': '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}", 
                           "% Passing Math": "{:.1%}", 
                           "% Passing Reading": "{:.1%}", 
                           "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.0619,83.9758,94.1%,97.0%,91.3%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.4183,83.8489,93.3%,97.3%,90.9%
Griffin High School,Charter,"1,468","$917,500",$625,83.3515,83.8168,93.4%,97.1%,90.6%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.2742,83.9895,93.9%,96.5%,90.6%
Pena High School,Charter,962,"$585,858",$609,83.8399,84.0447,94.6%,95.9%,90.5%


## Bottom Performing Schools (By Passing Rate)

In [6]:
bottom_5 = top_5.tail()
bottom_5 = bottom_5.sort_values('Overall Passing Rate')
bottom_5.style.format({'Total Students': '{:,}', 
                       "Total School Budget": "${:,}", 
                       "Per Student Budget": "${:.0f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",$637,76.8427,80.7447,66.4%,80.2%,53.0%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7118,81.158,66.0%,80.7%,53.2%
Huang High School,District,"2,917","$1,910,635",$655,76.6294,81.1827,65.7%,81.3%,53.5%
Hernandez High School,District,"4,635","$3,022,020",$652,77.2898,80.9344,66.8%,80.9%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.0725,80.9664,66.1%,81.2%,53.5%


## Math Scores by Grade

In [7]:
#creates grade level average math scores for each school 
ninth_math_score = stu_df.loc[stu_df['grade'] == '9th'].groupby('school')["math_score"].mean()
tenth_math_score = stu_df.loc[stu_df['grade'] == '10th'].groupby('school')["math_score"].mean()
eleventh_math_score = stu_df.loc[stu_df['grade'] == '11th'].groupby('school')["math_score"].mean()
twelfth_math_score = stu_df.loc[stu_df['grade'] == '12th'].groupby('school')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math_score,
        "10th": tenth_math_score,
        "11th": eleventh_math_score,
        "12th": twelfth_math_score
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School"

#show and format
math_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

In [8]:
ninth_reading_score = stu_df.loc[stu_df['grade'] == '9th'].groupby('school')["reading_score"].mean()
tenth_reading_score = stu_df.loc[stu_df['grade'] == '10th'].groupby('school')["reading_score"].mean()
eleventh_reading_score = stu_df.loc[stu_df['grade'] == '11th'].groupby('school')["reading_score"].mean()
twelfth_reading_score = stu_df.loc[stu_df['grade'] == '12th'].groupby('school')["reading_score"].mean()

#merges the reading score averages by school and grade together
reading_scores = pd.DataFrame({
        "9th": ninth_reading_score,
        "10th": tenth_reading_score,
        "11th": eleventh_reading_score,
        "12th": twelfth_reading_score
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School"

#format
reading_scores.style.format({'9th': '{:.1f}', 
                             "10th": '{:.1f}', 
                             "11th": "{:.1f}", 
                             "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

In [9]:
# create spending bins
bins = [0, 584.999, 614.999, 644.999, 999999]
group_spend = ['< $585', "$585 - 614", "$615 - 644", "> $644"]
merge_df['spending_bins'] = pd.cut(merge_df['budget']/merge_df['size'], bins, labels = group_spend)

#group by spending
by_spending = merge_df.groupby('spending_bins')

#calculations
avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = merge_df[merge_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = merge_df[merge_df['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
overall = merge_df[(merge_df['reading_score'] >= 70) & (merge_df['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()

            
# df build            
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_spend.index.name = "Per Student Budget"
scores_by_spend = scores_by_spend.reindex(group_spend)

#formating
scores_by_spend.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
< $585,83.4,84.0,93.7%,96.7%,90.6%
$585 - 614,83.5,83.8,94.1%,95.9%,90.1%
$615 - 644,78.1,81.4,71.4%,83.6%,60.3%
> $644,77.0,81.0,66.2%,81.1%,53.5%


## Scores by School Size


In [10]:
# create size bins
bins = [0, 999, 1999, 99999999999]
group_size = ["(<1000)", "(1000-2000)" , "(>2000)"]
merge_df['size_bins'] = pd.cut(merge_df['size'], bins, labels = group_size)

#group by spending
by_size = merge_df.groupby('size_bins')

#calculations 
avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = merge_df[merge_df['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
pass_read = merge_df[merge_df['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
overall = merge_df[(merge_df['reading_score'] >= 70) & (merge_df['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()

            
# df build            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_size)

#formating
scores_by_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
(<1000),83.8,83.8,94.0%,96.0%,90.1%
(1000-2000),83.4,83.4,93.6%,96.8%,90.6%
(>2000),77.5,77.5,68.7%,82.1%,56.6%


## Scores by School Type

In [11]:
# group by type of school
by_type = merge_df.groupby("type")

#calculations 
avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = merge_df[merge_df['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = merge_df[merge_df['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall = merge_df[(merge_df['reading_score'] >= 70) & (merge_df['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()

# df build            
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})
    
#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]
scores_by_type.index.name = "Type of School"


#formating
scores_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.4,93.7%,96.6%,90.6%
District,77.0,77.0,66.5%,80.9%,53.7%
